# Generate training data

### ideas not yet done

1. if a,b are labeled as same question pair, we can easily gen new training data by

        a. for c!=a  =>  c!=b
        b. for c==b && d==a  =>  c==d
        
2. Features:
        
        a. question length
        b. how common the words in question is (need a metric to measure)
        

In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import json

## Load data

In [2]:
df_train = pd.read_csv('../dataset/raw/train.csv', delimiter=',')
df_test = pd.read_csv('../dataset/raw/test.csv', delimiter=',')
df_train_size = 404290
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
all_questions = np.concatenate([np.array(df_train['question1']), np.array(df_train['question2'])])

## Parse words

Identify the words which are worth for embedding.

In [4]:
def clean_string(text):
    # Clean the text
    text = re.sub("what's", "what is ", text)
    text = re.sub("\'s", " ", text)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "cannot ", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am ", text)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub(" e.g.", " eg ", text)
    text = re.sub(" b.g.", " bg ", text)
    text = re.sub("(\d+)(kK)", "\g<1>000", text)
    text = re.sub("e-mail", "email", text)
    
    return text

def parse_single_words(question, idx=None):
    
    '''
    Usage:

        Use regex parse all single words without special characters and non-ascii terms

    Returns:

        List of string which is identified as a single word without special characters

    Testcases:

        # test first ten questions in dataset
        for i in range(10):
            print(i)
            print(all_questions[i])
            print(parse_single_words(all_questions[i]))
            
        # another good testcase
        print(parse_single_words(" '我是Hubert Lin, 這 code? 看起來還不錯ㄅ ? ha-ha-ha' "))

    '''
    
    ascii_only = False
    
    # identify special characters that separate words : (space) ' ! " ? @ ^ + * / . , ~ ( ) [ ] { } & | ` $ % = : ; < > -
    separator_include_ascii = '(?=[\s\'!"?@\^+*/\.,~\(\)\[\]\{\}\&\|`\$\%\=:;\<\>\-]|[^\x00-\x7F])'
    separator = '(?=[\s\'!"?@\^+*/\.,~\(\)\[\]\{\}\&\|`\$\%\=:;\<\>\-])'
    
    single_word = '[a-zA-Z0-9]+[a-zA-Z0-9_]*' # rule is at least one eng character or number
    
    try:
        question = clean_string(question)
        if not ascii_only:
            words = re.findall( single_word + separator_include_ascii + '|[^\x00-\x7F]' , question)

        elif ascii_only:
            words = re.findall( single_word + separator , question)
    except:
        idx = idx if idx<df_train_size else idx-df_train_size
        print('Got execption while handling' , idx, '-th question: ', question)
        print('There are two question pairs has empty string, got to deal with that')
        words = []
        
    return words

In [5]:
# Can't do this since return values are lists with different lengths, 
# this will cause error while np tries to transform all results into np array
#
# parse_single_words = np.vectorize(parse_single_words)
# all_questions_parsed = parse_single_words(all_questions)

all_questions_parsed = [parse_single_words(question, idx=i) for i,question in enumerate(all_questions)]

Got execption while handling 105780 -th question:  nan
There are two question pairs has empty string, got to deal with that
Got execption while handling 201841 -th question:  nan
There are two question pairs has empty string, got to deal with that


In [6]:
flatten_word_vec = np.concatenate(all_questions_parsed)
unique_word_vec, unique_word_count = np.unique(flatten_word_vec, return_counts=True)

#### check if word is in GloVe , if not , do some simple transformation. If this still doesn't work, trash the word.

In [10]:
def load_glove():
    embeddings_index = {}

    print('Loading from GloVe file on hard disk ... ')
    f = open('../dataset/raw/glove.840B.300d.txt', 'r', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except Exception as e:
            if e==KeyboardInterrupt:
                raise KeyBoardInterrupt
            print('exception word: ', values[0], ' ', values[1], ' ', values[2])
            continue
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index
    
glove = load_glove()

Loading from GloVe file on hard disk ... 
exception word:  .   .   .
exception word:  at   name@domain.com   0.0061218
exception word:  .   .   .
exception word:  to   name@domain.com   0.33865
exception word:  .   .   0.035974
exception word:  .   .   .
exception word:  email   name@domain.com   0.33529
exception word:  or   name@domain.com   0.48374
exception word:  contact   name@domain.com   0.016426
exception word:  Email   name@domain.com   0.37344
exception word:  on   name@domain.com   0.037295
exception word:  At   Killerseats.com   -0.13854
exception word:  by   name@domain.com   0.6882
exception word:  in   mylot.com   -0.18148
exception word:  emailing   name@domain.com   0.39173
exception word:  Contact   name@domain.com   0.14933
exception word:  at   name@domain.com   0.44321
exception word:  •   name@domain.com   -0.13288
exception word:  at   Amazon.com   -0.5275
exception word:  is   name@domain.com   -0.1197


In [11]:
glove_keys_lowercases = {key.lower():key for key in glove}
# approximately_mapping = {}

def is_glove_embeddable(word):
    
    if word in glove:
        return word
    
    if word.lower() in glove_keys_lowercases:
        return glove_keys_lowercases[word.lower()]
    
    # Wanna deal with 'un-' or 'in-' prefix cases here, but seems not so neccesary
    
#     for key in glove.keys():
#         if word in key
#             approximately_mapping
    
    
    return None

In [12]:
# Remove noise words, whom occurs less than threshold
# More words may increase performance, but increases size of word-embedding matrix
occurrence_threshold = 3
frequent_words = unique_word_vec[unique_word_count>=occurrence_threshold]

# let the GloVe do removing weird words for us
glove_handlable_words = np.array([is_glove_embeddable(word) for word in frequent_words])

selected_indexes = glove_handlable_words!=np.array([None])
selected_words = frequent_words[selected_indexes]
embedding_words_list = glove_handlable_words[selected_indexes]

print(len(unique_word_vec), ' unique words')
print(len(selected_words), 'words occurs more (or equal) than', occurrence_threshold , 'times')

json.dump(list(selected_words), open('../dataset/processed/selected_words.json', 'w'))

110183  unique words
47142 words occurs more (or equal) than 3 times


In [14]:
import random

trash_words = frequent_words[glove_handlable_words==np.array([None])]
' | '.join([unique_word_vec[random.randint(0,len(trash_words)-1)] for i in range(50)])

'1030 | 130k | 14443 | 20__ | 156cm | 12v | 11yrs | 11301 | 10170 | 1023 | 17th | 2185 | 11pm | 150ibs | 1611 | 10pm | 1m | 13000mAh | 10months | 20MP | 24075 | 18yr | 196lbs | 15k | 1700th | 1055 | 16GB | 12H143 | 19pencils | 11class | 13cm | 130K | 11b | 1976 | 100CE | 1903 | 12inch | 12801 | 140M | 1yr | 11 | 100kg | 0w | 00am | 12203 | 2160p | 1547 | 1vs1 | 23400 | 0n7'

## Generate encode/decode mapping

Encode the question into integers lists, this can reduce time consumption while we do word embedding on training phase

In [15]:
def build_enc_dec_mapping(words):
    """
    enc_map: voc --encode--> id
    dec_map: id --decode--> voc
    """
    
    enc_map, dec_map = {}, {}
    cannot_encode_words = []
    
    # add special control characters
    # <ST>    : start of question
    # <ED>    : end of question
    # <RARE>  : rare words occurrence is lower than threshold
    # <EMPTY> : no word in such position
    rare_characters = ['<RARE'+str(i)+'>' for i in range(100)] # 100 is just a number that I think large enough
    special_characters = [ '<ST>', '<ED>', '<EMPTY>'] + rare_characters
    sp_len = len(special_characters)
    for i,sp in enumerate(special_characters):
        enc_map[sp] = i
        dec_map[i] = sp
    
    # add our words into mapping
    for i,word in enumerate(words):
        if word in glove or is_glove_embeddable(word):
            enc_map[word] = sp_len+i
            dec_map[sp_len+i] = word
            
        else:
            cannot_encode_words.append(word)
            
    print(' | '.join(cannot_encode_words))
    
    return enc_map, dec_map

In [16]:
enc_map, dec_map = build_enc_dec_mapping(selected_words)

# save enc/decoding map to disk
# json.dump(enc_map, open('dataset/processed/enc_map.json', 'w'))
# json.dump(dec_map, open('dataset/processed/dec_map.json', 'w'))
pickle.dump(enc_map, open('../dataset/processed/enc_map.pkl', 'wb'))
pickle.dump(dec_map, open('../dataset/processed/dec_map.pkl', 'wb'))
vocab_size = len(dec_map)

In [17]:
def enc_question_pair(question_pair_list, enc_map, capsulate_st_ed=False):

    def enc_question(question):

        if type(question)!=str or question=="":
            if capsulate_st_ed:
                return [enc_map['<ST>'], enc_map['<ED>']]
            else:
                return []

        def map_wrod(word):
            if word in enc_map:
                return enc_map[word]
            elif word in rare_words:
                idx = np.where(np.array(rare_words)==word)[0][0]
                return enc_map['<RARE' + str(idx) + '>']
            else:
                rare_words.append(word)
                return enc_map['<RARE' + str(len(rare_words)-1) + '>']

        # identify special characters that separate words : (space) ' ! " ? @ ^ + * / . , ~ ( ) [ ] { } & | ` $ % = : ; < >  
        separator = '(?=[\s\'!"?@\^+*/\.,~\(\)\[\]\{\}\&\|`\$\%\=:;\<\>\-]|$)'
        single_word = '[^\s\-]+' # non-empty is enough here

        words_list = re.findall(single_word+separator, question)

        if capsulate_st_ed:
            return [enc_map['<ST>']] + [map_wrod(word) for word in words_list] + [enc_map['<ED>']] 
        else:
            return [map_wrod(word) for word in words_list]
        
    # This array is a local cache that identify those non-encoded rare words. 
    # We'll assign identical index (in this question pair) for each of rare words
    rare_words = []
    
    return enc_question(question_pair_list[0]) , enc_question(question_pair_list[1])
    
    
def dec_question(question, dec_map):
    return [dec_map[enc_value] for enc_value in question]

In [18]:
# testcase
question1 = 'glove is a good library howww awesomeeeee'
question2 = 'this work is awesomeeeee'
enc1, enc2 = enc_question_pair([question1, question2], enc_map)
dec1 = dec_question(enc1, dec_map)
dec2 = dec_question(enc2, dec_map)

print('Original question: ', question1, question2)
print('Encoded: ', enc1, enc2)
print('Decoded: ', dec1, dec2)

Original question:  glove is a good library howww awesomeeeee this work is awesomeeeee
Encoded:  [30119, 32666, 19889, 30182, 33650, 3, 4] [44304, 46811, 32666, 4]
Decoded:  ['glove', 'is', 'a', 'good', 'library', '<RARE0>', '<RARE1>'] ['this', 'work', 'is', '<RARE1>']


## Generate embedding matrix for GloVe

Warning:

Don't run this if not necessary. This takes about 10 minutes.

In [19]:
def generate_embedding_matrix(glove, words_list, dec_map, lang_dim):    
    
    out_vocab = []
    
    # prepare embedding matrix
    print('Load complete, start embedding ... ')
    embedding_matrix = np.random.rand(len(dec_map), lang_dim)
    for idx,word in enumerate(words_list):
        if word in glove.keys():
            embedding_matrix[idx] = glove[word]
        else:
            out_vocab.append(word)
            
    if len(out_vocab)>0:
        print('\nNot in pre-trained vocabulary list:\n', ','.join(out_vocab))
    else:
        print('\nAll words are embedded successfully')
    return embedding_matrix

In [20]:
dec_map = pickle.load(open('../dataset/processed/dec_map.pkl', 'rb'))

EMBEDDING_SIZE = 300
embedding_matrix = generate_embedding_matrix(glove, embedding_words_list, dec_map, EMBEDDING_SIZE)

pickle.dump(embedding_matrix, open('../dataset/processed/embedding_matrix.pkl', 'wb'))

Load complete, start embedding ... 

All words are embedded successfully


## Encode original training data frame and store it

In [21]:
enc_map = pickle.load(open('../dataset/processed/enc_map.pkl','rb'))

In [22]:
def preprocess_df(df, enc_map):
    
    def process_frame(frame):
        question_pair = [frame['question1'], frame['question2']]
        frame['question1'] , frame['question2'] = enc_question_pair(question_pair, enc_map)
        return frame
    
    return df.apply(process_frame, axis=1)

In [23]:
df_preprocessed = preprocess_df(df_train, enc_map)

# remove these two useless columns
del df_preprocessed['qid1']
del df_preprocessed['qid2']
# df_preprocessed.to_csv('../dataset/processed/processed_training_data.csv', index=False)
pickle.dump(df_preprocessed, open('../dataset/processed/processed_training_data.pkl', 'wb'))

df_preprocessed.head()

,id,question1,question2,is_duplicate
0,0,"[19354, 32666, 44207, 42906, 23164, 42906, 304...","[19354, 32666, 44207, 42906, 23164, 42906, 304...",0
1,1,"[19354, 32666, 44207, 43024, 36339, 10433, 3, ...","[19354, 46871, 30660, 31592, 44207, 9371, 3022...",0
2,2,"[8861, 23291, 8972, 31895, 44207, 42532, 36339...","[8861, 23291, 9512, 42532, 22036, 31896, 23164...",0
3,3,"[19400, 20710, 8972, 34717, 45992, 3, 8861, 23...","[7257, 44207, 39954, 46574, 5, 32666, 27075, 2...",0
4,4,"[19372, 36439, 27001, 31811, 46391, 39195, 3, ...","[19372, 29136, 46871, 43570, 31811, 40880, 6]",0


In [24]:
df_preprocessed = preprocess_df(df_test, enc_map)
# df_preprocessed.to_csv('./dataset/processed/processed_testing_data.csv', index=False)
pickle.dump(df_preprocessed, open('../dataset/processed/processed_testing_data.pkl', 'wb'))

df_preprocessed.head()

,test_id,question1,question2
0,0,"[8861, 27137, 44207, 17454, 14563, 31023, 970,...","[19400, 26642, 12141, 23989, 25254, 34103, 208..."
1,1,"[16669, 8972, 30757, 19889, 30559, 44818, 2146...","[8861, 35454, 25331, 27137, 30559, 44818, 5]"
2,2,"[19354, 23134, 32666, 44207, 22210, 46426, 444...","[19354, 47026, 41368, 35250, 44492, 4]"
3,3,"[19372, 29374, 36075, 3]","[19354, 29376, 4]"
4,4,"[8861, 3, 42827, 4]","[8861, 44216, 23291, 8972, 42827, 4]"


## Let's do a short experiment see if our embedding matrix works or not

Does vector : " London -> Paris " contains semantic relationship and can approximate relationship of " England -> France " ?

In [ ]:
from math import sqrt

print(enc_map['London'], enc_map['Paris'], enc_map['England'])

London = embedding_matrix[enc_map['London']]
Paris = embedding_matrix[enc_map['Paris']]
England = embedding_matrix[enc_map['England']]
target = Paris - London + England

distances = np.zeros(embedding_matrix.shape[0])
for i,vec in enumerate(embedding_matrix):
    distances[i] = sqrt(np.sum(np.square(vec-target)))

bests = np.argsort(distances)[:5]
for best in bests:
    print(dec_map[best], '  dist = ', distances[best])

It works !

## How to use this embedding matrix afterward (Example in Keras)

Note:

Since some weird words is not embeddable, so we random assign a random value vector to related word. Although this workaround might have some problems such as : the random assigned vector is probably being very close to other non-related word vector, which makes our model mis-understanding the meaning of such word. But at least our model has ability to identify if two words are same or not.

翻譯蒟蒻 (translation) :

因為我們選定的不少字不存在於 embedding file 裡面，而我們的處理方法是直接隨機給予一個 embedding vector ，這雖然會讓我們的 model 誤認為這些字與某些其他字有相關性，但至少這樣的做法讓我們的 model 有能力辨認兩個字是不是一樣的。

In [ ]:
# a short example how to embed encoded text into word vector

def model():
    
    '''This model example is for Keras'''

    # hyper-parameters that should be passed as function argument
    
    vocab_size = len(enc_map)
    vocab_dim = EMBEDDING_SIZE # in my case is 300
    text_length = 1 # how many words are we going to feed in one time

    
    '----------------------------------------------------------------------------'

    
    # embed our encoded question to embedded vector

    encoded_question_input = Input(shape=(text_length,))

    x = Embedding(output_dim = vocab_dim, 
                  input_dim = vocab_size, 
                  init = 'glorot_uniform',       # to be honest, I don't know what is this
                  input_length = text_length, 
                  weights = [embedding_matrix]   # our embedding_matrix
                 )(encoded_question_input)

    text_embedded = Reshape((vocab_dim*text_length,))(x)

    # maybe other input source

    other_input = . . . 
    other_input_dim = . . .

    # feed to RNN model

    x = merge([text_embedded, other_input], mode='concat', concat_axis=-1)
    x = Reshape((1, vocab_dim*text_length + other_input_dim ))(x)
    x = GRU(128)(x)
    out = Dense(vocab_size, activation='softmax')(x)
    
    # compile the model
    
    model = Model(input=[encoded_question_input, other_input], output=out)
    # choose objective and optimizer
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-3, clipnorm=clipnorm))
    
    return model